In [1]:
import json

### Path to Lung Ontology File

In [2]:
LOCAL_ONTOLOGY_FILE = '../Cell tissue test AMM.owl'

### Installed Imports

In [3]:
import ontospy

### Define functions

In [4]:
def get_onto_has_part_some(uri):
    sparql_has_part_some_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX : <http://www.semanticweb.org/am175/ontologies/2018/2/untitled-ontology-234#>
SELECT ?c WHERE {
    ?c rdfs:subClassOf ?restriction .
    ?restriction owl:onProperty :has_part ; owl:someValuesFrom ?p .
    VALUES ?p { <%s> }
}
""" % uri

    results = onto.query(sparql_has_part_some_query)
    
    return results

def recurse_has_part_some(uri):
    parts = get_onto_has_part_some(uri)
    
    parts_list = []
    
    for p in parts:
        sub_parts = recurse_has_part_some(p[0])
        if sub_parts is not None and len(sub_parts) > 0:
            parts_list.append(sub_parts)
    if uri is not None:
        return {uri.title(): parts_list}

### Load Ontology

In [5]:
onto = ontospy.Ontospy(uri_or_path=LOCAL_ONTOLOGY_FILE, rdf_format='xml')

In [6]:
onto.namespaces

[('',
  rdflib.term.URIRef('http://www.semanticweb.org/am175/ontologies/2018/2/untitled-ontology-234#')),
 ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('sh', rdflib.term.URIRef('http://www.w3.org/ns/shacl#')),
 ('skos', rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#')),
 ('vann', rdflib.term.URIRef('http://purl.org/vocab/vann/')),
 ('void', rdflib.term.URIRef('http://rdfs.org/ns/void#')),
 ('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'))]

### Get Ontology Proteins

In [7]:
sparql_proteins_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.semanticweb.org/am175/ontologies/2018/2/untitled-ontology-234#>
SELECT ?p WHERE {
    ?p rdfs:subClassOf :protein .
}
"""

onto_proteins = onto.query(sparql_proteins_query)

In [8]:
onto_proteins

[(rdflib.term.URIRef('http://www.semanticweb.org/am175/ontologies/2017/1/untitled-ontology-79#vimentin')),
 (rdflib.term.URIRef('http://www.semanticweb.org/am175/ontologies/2017/1/untitled-ontology-79#transcription_factor_SOX-9'))]

### Find related cells and tissues for proteins

In [9]:
onto_proteins

[(rdflib.term.URIRef('http://www.semanticweb.org/am175/ontologies/2017/1/untitled-ontology-79#vimentin')),
 (rdflib.term.URIRef('http://www.semanticweb.org/am175/ontologies/2017/1/untitled-ontology-79#transcription_factor_SOX-9'))]

In [10]:
probe_relations = []

for p_uri in onto_proteins:
    probe_relations.append(recurse_has_part_some(p_uri[0]))

In [11]:
print(json.dumps(probe_relations, indent=2))

[
  {
    "Http://Www.Semanticweb.Org/Am175/Ontologies/2017/1/Untitled-Ontology-79#Vimentin": [
      {
        "Http://Www.Semanticweb.Org/Am175/Ontologies/2017/1/Untitled-Ontology-79#Fibroblast": []
      }
    ]
  },
  {
    "Http://Www.Semanticweb.Org/Am175/Ontologies/2017/1/Untitled-Ontology-79#Transcription_Factor_Sox-9": [
      {
        "Http://Www.Semanticweb.Org/Am175/Ontologies/2017/1/Untitled-Ontology-79#Unclassified_Fibroblast": [
          {
            "Http://Www.Semanticweb.Org/Am175/Ontologies/2017/1/Untitled-Ontology-79#Interstitial_Tissue": []
          }
        ]
      }
    ]
  }
]
